In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("/sfs/qumulo/qhome/jtb3sud/PARCv2")
import PARC.data.data_meta_learning as data
#from parc import misc, metrics, visualization
from PARC.model import model_meta_learning as model


# Data pipeline

In [ ]:
# Get data and normalization
state_seq_whole, vel_seq_whole = data.clip_raw_data(sequence_length = 52, n_state_var = 3, purpose = "diff_training", folder_path = '/scratch/jtb3sud/meta_data/tnt/training', image_size = (128, 208))
state_seq_norm = data.data_normalization(state_seq_whole,3)
vel_seq_norm = data.data_normalization(vel_seq_whole,2)

# Training


### Stage 1: Differentiator training

In [ ]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,:3],vel_seq_norm[0][:,:,:,:2]))
dataset_label = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,-3:],vel_seq_norm[0][:,:,:,-2:]))
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 2192) 
dataset = dataset.batch(8)

In [ ]:
print(state_seq_whole.shape, vel_seq_whole.shape)

In [ ]:
print(state_seq_norm[0].shape, vel_seq_norm[0].shape)

In [ ]:
dataset

In [ ]:
tf.keras.backend.clear_session()
# step size needs to be changed to 1/total number of steps
#parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/60, solver = "rk4", mode = "differentiator_training")
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/52, solver = "rk4", mode = "differentiator_training", image_size = (128, 208))
parc.differentiator.load_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/em/parc2_diff_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.differentiator.save_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/tnt/parc2_diff_rk4.h5')

### Stage 2: Data-driven integration training

In [ ]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/52, solver = "rk4", mode = "integrator_training", image_size = (128, 208))
parc.integrator.load_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/em/parc2_int_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.integrator.save_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/tnt/parc2_int_rk4.h5')